In [2]:
import requests
import re
import nltk

from nltk import FreqDist
from nltk import word_tokenize

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
from urllib import request

In [3]:
#Página web de la cual se va a extraer la información
url='https://www.elespectador.com/economia/contraloria-pide-evitar-volatilidad-e-incertidumbre-al-hablar-de-mineria-y-energia/'
#Se envia solicitud a la página
resultado=requests.get(url)
#Se solicita el texo
content=resultado.text
soup=BeautifulSoup(content, 'lxml')

In [4]:
#Strip borrar espacios en blanco al inicio y al final 
#Separator borra saltos de línea 
box=soup.find('section',class_='false').get_text(strip=True,separator=' ')
title=soup.find('h1',class_='Title ArticleHeader-Title Title_article').get_text()
with open(f'{title}.txt','w') as file: # title + '.txt'
  file.write(box)

tokens=re.findall('\w+', box)
tokenizer=RegexpTokenizer('\w+')
tokens=tokenizer.tokenize(box)
tokens=[token.lower() for token in tokens]

In [5]:
def top_freqdst(tokens,n=100):
    # Listado de stopwords
    stops = stopwords.words('spanish')

    # tokens filtrados de stopwords
    filtered_tokens = [token for token in tokens if token not in stops]

    #freqdist de tokens
    f_dst = FreqDist(filtered_tokens)
    top_n = f_dst.most_common(n)

    return dict(top_n)



In [ ]:
def atributos(tokens):
    atrib = {}

    atrib['top_freqdist'] = top_freqdst(tokens,n=100)

    return atrib
